In [1]:
import streamlit as st
import pandas as pd
import numpy as np
import warnings
import plotly.express as px

In [ ]:
# 二级报警阈值（V）
TH_LEVEL2 = 2.95
# 强制充电请求阈值（V）
TH_FORCE_CHG = 2.85
# 一级报警阈值，断开接触器（V）
TH_LEVEL1 = 2.7
# 微断脱扣阈值（V）
TH_OFF_QF = 2.6
# 储能柜功率(kW)
ESS_POWER = 125
# 系统响应时间（s）
SYS_RESP_T = 10
# 待机功耗（W）
STANDBY_P = 100
# 电池串数
BATT_SERIES = 260
# 系统损耗
SYS_LOSS = 0.7

In [3]:
df = pd.read_excel('SOC-OCV与放电曲线.xlsx', sheet_name='亿纬314Ah0.5P放电曲线1')

In [19]:
se_th_level2 = df[df['实际电压(V)'] < TH_LEVEL2].iloc[0]
se_th_level2['名称'] = '二级报警阈值'
se_th_level2['阈值'] = TH_LEVEL2

se_th_force_chg = df[df['实际电压(V)'] < TH_FORCE_CHG].iloc[0]
se_th_force_chg['名称'] = '强制充电请求阈值'
se_th_force_chg['阈值'] = TH_FORCE_CHG

se_th_level1 = df[df['实际电压(V)'] < TH_LEVEL1].iloc[0]
se_th_level1['名称'] = '一级报警阈值'
se_th_level1['阈值'] = TH_LEVEL1

se_th_off_qf = df[df['实际电压(V)'] < TH_OFF_QF].iloc[0]
se_th_off_qf['名称'] = '微断脱扣阈值'
se_th_off_qf['阈值'] = TH_OFF_QF


In [27]:
# 停止放电到强充标志置位时间（S）T1
T1 = BATT_SERIES*(se_th_force_chg['能量(Wh)'] - se_th_level2['能量(Wh)']) / 1000 * 3600 / ESS_POWER

T2 = (BATT_SERIES*(se_th_level1['能量(Wh)'] - se_th_level2['能量(Wh)']) - ESS_POWER * 1000 * SYS_RESP_T/3600) / STANDBY_P * SYS_LOSS

In [28]:
print(f"停止放电到强充标志置位时间T1: {T1:.0f}秒\n停止放电到断开主回路接触器时间T2: {T2:.0f}小时")

停止放电到强充标志置位时间T1: 36秒
停止放电到断开主回路接触器时间T2: 23小时


In [25]:
df_th_val = pd.concat([se_th_level2, se_th_force_chg, se_th_level1, se_th_off_qf], axis=1)

In [26]:
df_th_val.T

,实际电压(V),容量(Ah),能量(Wh),名称,阈值
7426,2.8993,322.886,1036.984,二级报警阈值,2.9
7460,2.8494,324.5372,1041.73,强制充电请求阈值,2.85
7528,2.6971,327.9536,1051.229,一级报警阈值,2.7
7553,2.5992,329.27,1054.719,微断脱扣阈值,2.6
